In [1]:
import math
import numpy as np

from anguilla.fitness.benchmark import Sphere, SumSquares

A translation of the Matlab code from [2016:cma-es-tutorial].

In [2]:
def cmaes(fitness_function, n=5, sigma=0.5, stopfitness=1e-10, stopeval=1e5, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    # Strategy parameters for selection
    lambda_ = 4 + math.floor(3. * math.log(n))
    mu = lambda_ // 2
    weights = math.log(mu + 0.5) - np.log1p(np.arange(mu))
    weights /= np.sum(weights)
    mu_eff = np.sum(weights) ** 2 / np.sum(weights * weights)

    xmean = fitness_function.propose_initial_point(n)

    # Strategy parameters for adaptation
    n_f = float(n)
    cc = (4. + mu_eff / n_f) / (n_f + 4. + 2. * mu_eff / n_f)
    cs = (mu_eff  + 2.) / (n + mu_eff + 5.)
    c1 = 2. / ((n_f + 1.3) ** 2 + mu_eff)
    cmu = 2. * (mu_eff - 2. + 1. / mu_eff) / ((n_f + 2.) ** 2 + 2. * mu_eff / 2)
    damps = 1. + 2. * max(0., math.sqrt((mu_eff - 1.) / (n_f + 1.)) - 1.)

    # Initialization
    pc = np.zeros(n)
    ps = np.zeros(n)
    B = np.eye(n)
    D = np.eye(n)
    C = (B @ D) @ (B @ D).T
    eigeneval = 0
    exp_chi = math.sqrt(n) * (1. - 1. / (4. * n) + 1. / (21. * n * n))

    # Generation loop
    counteval = 0
    while counteval < stopeval:
        # Generate and evaluate lambda offspring
        #Z = np.random.standard_normal(size=(lambda_, n))
        #BD = B @ D
        #X = xmean + sigma * (Z @ BD.T)
        BD = B @ D
        Z = np.empty((n, lambda_))
        X = np.empty((n, lambda_))
        fitness_values = np.empty(lambda_)
        for i in range(lambda_):
            Z[:, i] = rng.standard_normal(size=n)
            X[:, i] = xmean + sigma * (BD @ Z[:, i])
            fitness_values[i] = fitness_function(X[:, i])
            counteval += 1

        # Selection
        ranked_indices = np.argsort(fitness_values)
        selected_indices = ranked_indices[:mu]

        # Recombination
        #xmean =  weights @ X[selected_indices]
        #zmean =  weights @ Z[selected_indices]

        xmean = X[:, selected_indices] @ weights
        zmean = Z[:, selected_indices] @ weights
        assert(xmean.shape == (n, ))
    
        # Cumulation
        ps = (1. - cs) * ps \
            + (math.sqrt(cs * (2 - cs) * mu_eff)) \
            * (B @ zmean)
        assert ps.shape == (n,)

        hsig_lhs = np.linalg.norm(ps) / math.sqrt(1. - (1. - cs) ** (2. * counteval / lambda_)) / exp_chi
        hsig_rhs = 1.4 + 2. / (n_f + 1.)
        hsig = 1. if hsig_lhs < hsig_rhs else 0.

        pc = (1. - cc) * pc + hsig * math.sqrt(cc * (2. - cc) * mu_eff) * (BD @ zmean)
        assert pc.shape == (n,)

        # Adapt covariance matrix
        BDZ = BD @ Z[:, selected_indices]
        C = (1. - c1 - cmu) * C \
            + c1 * (np.outer(pc, pc)
                + (1. - hsig) * cc * (2-cc) * C) \
            + cmu * (BDZ @ np.diag(weights) @ BDZ.T)
        assert C.shape == (n, n)

        # Adapt step size
        sigma = sigma * math.exp((cs / damps) * (np.linalg.norm(ps) / exp_chi - 1.))

        # Update B and D from C
        if counteval - eigeneval > lambda_  / (c1 + cmu) / n_f / 10.:
            eigeneval = counteval
            C = np.triu(C) + np.triu(C, 1).T
            Dsq, B = np.linalg.eigh(C)
            D = np.diag(np.sqrt(Dsq))

        # Good enough fitness
        if fitness_values[ranked_indices[0]] <= stopfitness:
            print('Good fitness')
            break

        # Flat fitness
        other_index = math.ceil(0.7 * lambda_)    
        if fitness_values[ranked_indices[0]] == fitness_values[ranked_indices[other_index]]:
            print('Flat fitness')
            sigma = sigma * math.exp(0.2 + cs / damps)
        
        print(f'{counteval}: {fitness_values[ranked_indices[0]]}')

    return X[:, ranked_indices[0]]

In [3]:
sphere = Sphere()
cmaes(sphere, n=3, sigma=0.3)

7: 4.249589768070005
14: 4.945119137713975
21: 0.9643078485217149
28: 0.007576128334343472
35: 0.4294982451442918
42: 0.2747711327203762
49: 0.3798387275612437
56: 0.7276128015534487
63: 0.743665632022367
70: 0.9754510938402857
77: 1.0637453088761475
84: 1.36237068962775
91: 0.6919502146793386
98: 0.5216874702100008
105: 0.29636411688430236
112: 0.3752200134716559
119: 0.17311980584232656
126: 0.16542742621944292
133: 0.030382711889029492
140: 0.12056507651618613
147: 0.03808119044245091
154: 0.04384647442759544
161: 0.06303861749902478
168: 0.003323252279038339
175: 0.020602980257992388
182: 0.017924572754122722
189: 0.00795503292619434
196: 0.005009500483150054
203: 4.8257338166340554e-05
210: 0.0013540953226931077
217: 0.0015356306929429123
224: 0.0029554332046123452
231: 0.0009187056000410443
238: 9.800957472781867e-05
245: 0.0008237195549124147
252: 0.0005620751217267743
259: 9.746177193097019e-05
266: 4.4717591323004334e-05
273: 4.1263859237548395e-05
280: 7.968412118389894e-05
2

array([-2.25939476e-06, -1.19727781e-06,  5.61511762e-06])

In [4]:
sum_squares = SumSquares()
cmaes(sum_squares, n=4, sigma=0.3)

8: 1.8444083805959837
16: 0.8885081923432125
24: 0.4386944204002752
32: 1.054371010430749
40: 1.3278581472260182
48: 0.4072351002446175
56: 0.6761790983814029
64: 0.21410880342688257
72: 1.161425195174165
80: 0.8557185567732593
88: 0.8608600152285921
96: 0.4179734099672736
104: 0.33241064114888097
112: 0.13672711194956577
120: 0.03454939619132344
128: 0.0710679310972795
136: 0.15395826426361203
144: 0.027430593593928368
152: 0.01709053290147959
160: 0.020522352005687215
168: 0.02107757851308677
176: 0.018388955282663166
184: 0.019414116030801835
192: 0.013151614947749034
200: 0.005924188173216293
208: 0.004922070588237025
216: 0.005639892030548509
224: 0.002406989877018563
232: 0.0014859976625330924
240: 0.0011150989961573913
248: 0.0010576289916830962
256: 0.00042649602026147984
264: 0.00043846209407826425
272: 0.00014570019174363255
280: 0.00023095986513686946
288: 4.606415769597916e-05
296: 0.0003638805354377424
304: 0.00018003400160378514
312: 4.4168735252105796e-05
320: 6.78066344

array([-6.40443387e-06,  3.40507053e-06, -2.13093698e-06,  2.77209408e-07])